In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Code adapted from Professor Bargal's CS440 class challenge file



In [2]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#os.environ['OMP_NUM_THREADS'] = '1'

tf.__version__

'2.7.0'

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
import PIL
image = PIL.Image.open("/content/gdrive/MyDrive/CS583/Testing Images/Japanese/Japanese_3.png") 
width, height = image.size
print(width, height)

216 144


In [5]:
#DATA_LIST = os.listdir('/content/gdrive/MyDrive/CS583/Training Images')
#test_list = os.listdir('/content/gdrive/MyDrive/CS583/Testing Images')
#DATASET_PATH  = '/content/gdrive/MyDrive/CS583/Training Images'
#TEST_DIR =  '/content/gdrive/MyDrive/CS583/Testing Images'
DATA_LIST = os.listdir('/content/gdrive/MyDrive/CS583/Alternative Train')
test_list = os.listdir('/content/gdrive/MyDrive/CS583/Alternative Test')
#test_list = os.listdir('/content/gdrive/MyDrive/CS583/Second Alternative Test')
DATASET_PATH  = '/content/gdrive/MyDrive/CS583/Alternative Train'
#TEST_DIR =  '/content/gdrive/MyDrive/CS583/Alternative Test'
TEST_DIR =  '/content/gdrive/MyDrive/CS583/Second Alternative Test'
IMAGE_SIZE    = (216, 144)
NUM_CLASSES   = len(DATA_LIST)
num_test = len(test_list)
BATCH_SIZE    = 64 
NUM_EPOCHS    = 29
LEARNING_RATE = 0.0003

In [6]:
print(DATA_LIST)
print(test_list)

['Japanese', 'Chinese', 'English', 'French', 'Korean', 'Spanish']
['Korean', 'Japanese', 'French', 'Chinese', 'English', 'Spanish']


In [7]:
test_list

['Korean', 'Japanese', 'French', 'Chinese', 'English', 'Spanish']

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,featurewise_center = True,
                                   featurewise_std_normalization = True,width_shift_range=0.2,
                                   height_shift_range=0.2,shear_range=0.25,zoom_range=0.1,
                                   zca_whitening = False,channel_shift_range = 20,
                                   horizontal_flip = False,vertical_flip = True,
                                   validation_split = 0.2,fill_mode='constant')

In [9]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH,target_size=IMAGE_SIZE,
                                                  shuffle=True,batch_size=BATCH_SIZE,
                                                  subset = "training",seed=42,
                                                  class_mode="categorical")

valid_batches = train_datagen.flow_from_directory(DATASET_PATH,target_size=IMAGE_SIZE,
                                                  shuffle=True,batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,class_mode="categorical")

Found 1832 images belonging to 6 classes.
Found 457 images belonging to 6 classes.


In [10]:
train_batches.labels

array([0, 0, 0, ..., 5, 5, 5], dtype=int32)

In [11]:
train_batches.class_indices

{'Chinese': 0,
 'English': 1,
 'French': 2,
 'Japanese': 3,
 'Korean': 4,
 'Spanish': 5}

In [12]:
#from keras.applications.resnet_v2 import ResNet101V2
#sequential_1 = tf.keras.models.Sequential(ResNet101V2(include_top=False, weights="imagenet", input_shape=(216,144,3)))
#for layer in sequential_1.layers:
#    layer.trainable = False

171327488/171317808 [==============================] - 2s 0us/step


In [13]:
sequential_1 = tf.keras.models.Sequential()
sequential_1.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(216,144,3)))
#sequential_1.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
sequential_1.add(tf.keras.layers.MaxPooling2D(
    pool_size=(3, 3), strides=None, padding="same"))
sequential_1.add(tf.keras.layers.Flatten())
sequential_1.add(tf.keras.layers.Dense(256, activation='relu'))
sequential_1.add(tf.keras.layers.Dropout(0.25))
sequential_1.add(tf.keras.layers.Dense(6)) # 7 when the dataset is the original, 6 with the Tatoeba dataset
sequential_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 7, 5, 2048)        42626560  
                                                                 
 conv2d (Conv2D)             (None, 7, 5, 32)          589856    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 3, 2, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 256)               49408     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                        

In [14]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size

sequential_1.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
model = sequential_1.fit(train_batches, validation_data=valid_batches, epochs=NUM_EPOCHS, steps_per_epoch=STEP_SIZE_TRAIN, 
                 validation_steps = STEP_SIZE_VALID)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/29
28/28 [==============================] - 361s 13s/step - loss: 2.1779 - accuracy: 0.2393 - val_loss: 1.5571 - val_accuracy: 0.2969
Epoch 2/29
28/28 [==============================] - 29s 1s/step - loss: 1.3461 - accuracy: 0.4259 - val_loss: 1.2468 - val_accuracy: 0.5000
Epoch 3/29
28/28 [==============================] - 29s 1s/step - loss: 1.0142 - accuracy: 0.5979 - val_loss: 1.0763 - val_accuracy: 0.5938
Epoch 4/29
28/28 [==============================] - 29s 1s/step - loss: 0.8551 - accuracy: 0.6357 - val_loss: 0.9933 - val_accuracy: 0.6205
Epoch 5/29
28/28 [==============================] - 29s 1s/step - loss: 0.7487 - accuracy: 0.6861 - val_loss: 0.8386 - val_accuracy: 0.6786
Epoch 6/29
28/28 [==============================] - 29s 1s/step - loss: 0.6503 - accuracy: 0.7381 - val_loss: 0.8026 - val_accuracy: 0.7121
Epoch 7/29
28/28 [==============================] - 29s 1s/step - loss: 0.6323 - accuracy: 0.7523 - val_loss: 0.6687 - val_accuracy: 0.7321
Epoch 8/29
28/28 [

In [15]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

eval_generator = test_datagen.flow_from_directory(TEST_DIR,target_size=IMAGE_SIZE,
                                                  batch_size=1,shuffle=True,seed=42,class_mode="categorical")
eval_generator.reset()
print(len(eval_generator))
print(eval_generator.class_indices)
x = sequential_1.evaluate_generator(eval_generator,steps = np.ceil(len(eval_generator)),
                           use_multiprocessing = False,verbose = 1,workers=1)
print('Test loss:' , x[0])
print('Test accuracy:',x[1])

Found 2401 images belonging to 6 classes.
2401
{'Chinese': 0, 'English': 1, 'French': 2, 'Japanese': 3, 'Korean': 4, 'Spanish': 5}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  if __name__ == '__main__':


2401/2401 [==============================] - 400s 166ms/step - loss: 2.9595 - accuracy: 0.2220
Test loss: 2.9595024585723877
Test accuracy: 0.22199083864688873
